In [ ]:
from google.colab import auth
from google.cloud import storage
from typing import List

auth.authenticate_user()

In [12]:
! gcloud auth login


You are running on a Google Compute Engine virtual machine.
It is recommended that you use service accounts for authentication.

You can run:

  $ gcloud config set account `ACCOUNT`

to switch accounts if necessary.

Your credentials may be visible to others with access to this
virtual machine. Are you sure you want to authenticate with
your personal account?

Do you want to continue (Y/n)?  y

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=9selettxhNuAcOYNnV6AD

In [ ]:
! gcloud auth application-default login

In [3]:
! gcloud config set project dl4h-final-project-383605

Updated property [core/project].


In [6]:
project_id = "dl4h-final-project-383605"

storage_client = storage.Client(project=project_id)
bucket = storage_client.bucket("mimiciii-1.4.physionet.org")
for blob in bucket.list_blobs():
  if "CHARTEVENTS" in blob.name:
    continue
  blob.download_to_filename(blob.name)

In [7]:
# Extract all the files
! gunzip *.gz

In [1]:
! pip install pyhealth
from pyhealth.datasets import MIMIC3Dataset
from pyhealth.data import Patient, Visit
import pandas as pd
from pyhealth.datasets import split_by_patient, get_dataloader, BaseDataset
from pyhealth.models import Transformer, BaseModel
from pyhealth.trainer import Trainer
from pyhealth.metrics.binary import binary_metrics_fn
import torch
import torch.nn as nn
import numpy as np
from typing import Tuple, List, Dict, Optional
import functools

data_root = "/Users/cyg1122/Desktop/school/dl4h/mimic3/physionet.org/files/mimiciii/1.4/"


[notice] A new release of pip available: 22.3.1 -> 23.1
[notice] To update, run: pip install --upgrade pip


/Users/cyg1122/PycharmProjects/dl4h-final-project/venv/lib/python3.8/site-packages/pyhealth/trainer.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


In [2]:
mimic3_ds = MIMIC3Dataset(
        root=data_root,
        tables=["DIAGNOSES_ICD", "PROCEDURES_ICD"],
        dev=True
)

Mapping codes: 100%|██████████| 1000/1000 [00:00<00:00, 164160.63it/s]


In [3]:
mimic3_ds.stat()


Statistics of base dataset (dev=True):
	- Dataset: MIMIC3Dataset
	- Number of patients: 1000
	- Number of visits: 1295
	- Number of visits per patient: 1.2950
	- Number of events per visit in DIAGNOSES_ICD: 9.3544
	- Number of events per visit in PROCEDURES_ICD: 4.3351



'\nStatistics of base dataset (dev=True):\n\t- Dataset: MIMIC3Dataset\n\t- Number of patients: 1000\n\t- Number of visits: 1295\n\t- Number of visits per patient: 1.2950\n\t- Number of events per visit in DIAGNOSES_ICD: 9.3544\n\t- Number of events per visit in PROCEDURES_ICD: 4.3351\n'

In [4]:
# Find all diagnoses codes with less than five occurrences

all_diag_codes = []
all_proc_codes = []
for patient_id, patient in mimic3_ds.patients.items():
  for i in range(len(patient)):
    visit: Visit = patient[i]
    conditions = visit.get_code_list(table="DIAGNOSES_ICD")
    procedures = visit.get_code_list(table="PROCEDURES_ICD")
    all_diag_codes.extend(conditions)
    all_proc_codes.extend(procedures)

codes = pd.Series(all_diag_codes)
diag_code_counts = codes.value_counts()
filtered_diag_codes = diag_code_counts[diag_code_counts > 4].index.values
num_unique_diag_codes = len(filtered_diag_codes)

unique_proc_codes = list(set(all_proc_codes))
num_unique_proc_codes = len(unique_proc_codes)

In [5]:
proc_code_to_index_map = {}
diag_code_to_index_map = {}

index = 0
for proc_code in unique_proc_codes:
    proc_code_to_index_map[proc_code] = index
    index += 1

index = 0
for diag_code in filtered_diag_codes:
    diag_code_to_index_map[diag_code] = index
    index += 1

In [6]:
def proc_code_to_onehot(code, n_codes, code_to_index_map):
    index = code_to_index_map[code]
    vec = np.zeros(n_codes)
    vec[index] = 1
    return vec

In [7]:
# Define the tasks

def readmission_prediction_mimic3_fn_dx(patient: Patient, time_window=30):
    if len(patient) < 2:
        return []

    samples = []

    # we will drop the last visit
    for i in range(len(patient) - 1):
        first_visit: Visit = patient[0]
        current_visit: Visit = patient[i]
        next_visit: Visit = patient[i + 1]

        # get time difference between current visit and next visit
        time_diff_from_last_visit = (next_visit.encounter_time - current_visit.encounter_time).days
        time_diff_from_first_visit = (current_visit.encounter_time - first_visit.encounter_time).days
        readmission_label = 1 if time_diff_from_last_visit < time_window else 0

        # get num days since first visit

        conditions = [c for c in current_visit.get_code_list(table="DIAGNOSES_ICD") if c in filtered_diag_codes]
        # exclude: visits without condition, procedure, or drug code
        if len(conditions) == 0:
            continue
        samples.append(
            {
                "visit_id": current_visit.visit_id,
                "patient_id": patient.patient_id,
                "conditions": conditions,
                # "days_since_first_visit": [time_diff_from_first_visit],
                "label": readmission_label,
            }
        )
    # no cohort selection
    return samples

def readmission_prediction_mimic3_fn_dxtx(patient: Patient, time_window=30):
    if len(patient) < 2:
        return []

    samples = []

    # we will drop the last visit
    for i in range(len(patient) - 1):
        first_visit: Visit = patient[0]
        current_visit: Visit = patient[i]
        next_visit: Visit = patient[i + 1]

        # get time difference between current visit and next visit
        time_diff = (next_visit.encounter_time - current_visit.encounter_time).days
        time_diff_from_first_visit = (current_visit.encounter_time - first_visit.encounter_time).days
        readmission_label = 1 if time_diff < time_window else 0

        conditions = [c for c in current_visit.get_code_list(table="DIAGNOSES_ICD") if c in filtered_diag_codes]
        procedures = current_visit.get_code_list(table="PROCEDURES_ICD")
        # exclude: visits without condition, procedure, or drug code
        if len(conditions) * len(procedures) == 0:
            continue
        samples.append(
            {
                "visit_id": current_visit.visit_id,
                "patient_id": patient.patient_id,
                "conditions": conditions,
                "procedures": procedures,
                # "days_since_first_visit": [time_diff_from_first_visit],
                "label": readmission_label,
            }
        )
    # no cohort selection
    return samples

In [8]:
mimic3_dx = mimic3_ds.set_task(task_fn=readmission_prediction_mimic3_fn_dx)
mimic3_dxtx = mimic3_ds.set_task(task_fn=readmission_prediction_mimic3_fn_dxtx)

Generating samples for readmission_prediction_mimic3_fn_dx: 100%|██████████| 1000/1000 [00:00<00:00, 31526.88it/s]
Generating samples for readmission_prediction_mimic3_fn_dxtx: 100%|██████████| 1000/1000 [00:00<00:00, 32165.89it/s]


In [9]:
print(mimic3_dx.stat())

Statistics of sample dataset:
	- Dataset: MIMIC3Dataset
	- Task: readmission_prediction_mimic3_fn_dx
	- Number of samples: 293
	- Number of patients: 168
	- Number of visits: 293
	- Number of visits per patient: 1.7440
	- conditions:
		- Number of conditions per sample: 10.5324
		- Number of unique conditions: 419
		- Distribution of conditions (Top-10): [('4019', 98), ('4280', 97), ('42731', 76), ('41401', 63), ('5849', 58), ('5856', 57), ('51881', 46), ('486', 43), ('5990', 42), ('25000', 42)]
	- label:
		- Number of label per sample: 1.0000
		- Number of unique label: 2
		- Distribution of label (Top-10): [(1, 149), (0, 144)]
Statistics of sample dataset:
	- Dataset: MIMIC3Dataset
	- Task: readmission_prediction_mimic3_fn_dx
	- Number of samples: 293
	- Number of patients: 168
	- Number of visits: 293
	- Number of visits per patient: 1.7440
	- conditions:
		- Number of conditions per sample: 10.5324
		- Number of unique conditions: 419
		- Distribution of conditions (Top-10): [('401

In [10]:
print(mimic3_dxtx.stat())

Statistics of sample dataset:
	- Dataset: MIMIC3Dataset
	- Task: readmission_prediction_mimic3_fn_dxtx
	- Number of samples: 262
	- Number of patients: 154
	- Number of visits: 262
	- Number of visits per patient: 1.7013
	- conditions:
		- Number of conditions per sample: 10.5878
		- Number of unique conditions: 412
		- Distribution of conditions (Top-10): [('4280', 86), ('4019', 85), ('42731', 69), ('41401', 60), ('5849', 53), ('5856', 51), ('51881', 45), ('25000', 38), ('486', 36), ('5990', 36)]
	- procedures:
		- Number of procedures per sample: 4.6908
		- Number of unique procedures: 273
		- Distribution of procedures (Top-10): [('3893', 101), ('9904', 65), ('3995', 63), ('9604', 51), ('966', 48), ('9672', 37), ('9671', 34), ('3891', 30), ('8856', 27), ('9915', 23)]
	- label:
		- Number of label per sample: 1.0000
		- Number of unique label: 2
		- Distribution of label (Top-10): [(1, 137), (0, 125)]
Statistics of sample dataset:
	- Dataset: MIMIC3Dataset
	- Task: readmission_predic

In [11]:
# define the dataloaders
def create_dataloaders(dataset, split=[0.8, 0.1, 0.1]):
    train, val, test = split_by_patient(dataset, split)

    # obtain train/val/test dataloader, they are <torch.data.DataLoader> object
    train_loader = get_dataloader(train, batch_size=32, shuffle=True)
    val_loader = get_dataloader(val, batch_size=32, shuffle=False)
    test_loader = get_dataloader(test, batch_size=32, shuffle=False)
    return train_loader, val_loader, test_loader

train_loader_dx, val_loader_dx, test_loader_dx = create_dataloaders(mimic3_dx)
train_loader_dxtx, val_loader_dxtx, test_loader_dxtx = create_dataloaders(mimic3_dxtx)

In [55]:
# Define the models
from enum import Enum

class MaskDirection(Enum):
    FORWARD = 'forward'
    BACKWARD = 'backward'
    DIAGONAL = 'diagonal'
    NONE = 'none'

class MaskedLayerNorm(nn.Module):
    def __init__(self, normalized_shape: int):
        super().__init__()
        self.gamma = nn.parameter.Parameter(torch.randn(normalized_shape))
        self.beta = nn.parameter.Parameter(torch.randn(normalized_shape))
        self.eps = 1e-5
        self.normalized_shape = normalized_shape

    def forward(self, x: torch.Tensor, key_padding_mask: Optional[torch.Tensor] = None):
        print(x.shape)
        print(key_padding_mask.shape)
        n = torch.sum(torch.ones_like(x) * (~key_padding_mask).int().unsqueeze(-1).expand(-1, -1, self.normalized_shape), dim=-1)
        print(n.shape)
        print(n)

        # expected_val = torch.nanmean(x, dim=1)
        # variance = torch.nansum(())

# todo: add temporal encoding
# todo: implement layer normalization
# todo: implement fully connected network structure and linear layer activation functions
class MaskEnc(nn.Module):
    def __init__(self, embedding_dim: int, num_heads: int, dropout: float = 0.1, batch_first: bool = True, temporal_mask_direction: MaskDirection = MaskDirection.NONE):
        super().__init__()

        self.embedding_dim = embedding_dim
        self.temporal_mask_direction = temporal_mask_direction

        self.attention = nn.MultiheadAttention(
            embed_dim=embedding_dim,
            num_heads=num_heads,
            dropout=dropout,
            batch_first=batch_first
        )

        # self.masked_layer_norm1 = MaskedLayerNorm(embedding_dim)
        # self.masked_layer_norm2 = MaskedLayerNorm(embedding_dim)

        self.fc = nn.Linear(embedding_dim, embedding_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)

    def forward(self, x: torch.Tensor, key_padding_mask: Optional[torch.Tensor] = None):
        attn_mask = self._make_temporal_mask(x.shape[1])

        attn_output, attn_output_weights = self.attention(x, x, x, key_padding_mask=key_padding_mask, attn_mask=attn_mask)
        attn_output = torch.nan_to_num(attn_output, nan=-1e9)

        # x = self.masked_layer_norm1(x + attn_output, key_padding_mask)
        # x = self.masked_layer_norm2(x + self.fc(x), key_padding_mask)

        x = self.dropout(self.relu(self.fc(attn_output)))
        return x

    def _make_temporal_mask(self, n: int) -> Optional[torch.Tensor]:
        if self.temporal_mask_direction == MaskDirection.NONE:
            return None

        mask = torch.ones(n,n)
        if self.temporal_mask_direction == MaskDirection.FORWARD:
            mask = torch.tril(mask)
        if self.temporal_mask_direction == MaskDirection.BACKWARD:
            mask = torch.triu(mask)
        if self.temporal_mask_direction == MaskDirection.DIAGONAL:
            mask = mask.fill_diagonal_(0)

        return mask.bool()

class BiteNet(nn.Module):
    def __init__(self, embedding_dim: int, output_dim: int, num_heads: int, dropout: float = 0.1, batch_first: bool = True):
        super().__init__()

        def _make_mask_enc_block(temporal_mask_direction: MaskDirection = MaskDirection.NONE):
            return MaskEnc(
                embedding_dim = embedding_dim,
                num_heads = num_heads,
                dropout = dropout,
                batch_first = batch_first,
                temporal_mask_direction = temporal_mask_direction
            )

        self.code_level_attn = _make_mask_enc_block(MaskDirection.DIAGONAL)
        self.visit_level_attn_forward = _make_mask_enc_block(MaskDirection.FORWARD)
        self.visit_level_attn_backward = _make_mask_enc_block(MaskDirection.BACKWARD)
        self.fc1 = nn.Linear(2*embedding_dim, output_dim)
        self.fc2 = nn.Linear(embedding_dim, output_dim)
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(dropout)
        self.relu = nn.ReLU()

    def forward(self, x: torch.Tensor, key_padding_mask: Optional[torch.Tensor] = None) -> torch.Tensor:
        code_attn = self.code_level_attn(x, key_padding_mask)

        u_fw = self.visit_level_attn_forward(code_attn, key_padding_mask)
        u_fw = u_fw.nansum(dim=1).squeeze()

        u_bw = self.visit_level_attn_backward(code_attn, key_padding_mask)
        u_bw = u_bw.nansum(dim=1).squeeze()

        u_bi = torch.cat([u_fw, u_bw], dim=-1)
        u_bi = torch.nan_to_num(u_bi)
        s = self.relu(self.fc1(u_bi))
        # logits = self.fc2(s)
        return s

class PyHealthBiteNet(BaseModel):
    def __init__(self, dataset: BaseDataset, feature_keys: List[str], label_key: str, mode: str,
                 embedding_dim: int = 128, num_heads: int = 8, dropout: float = 0.1, batch_first: bool = True, **kwargs):
        super().__init__(dataset, feature_keys, label_key, mode)

        # Any BaseModel should have these attributes, as functions like add_feature_transform_layer uses them
        self.feat_tokenizers = {}
        self.embeddings = nn.ModuleDict()
        self.linear_layers = nn.ModuleDict()
        self.label_tokenizer = self.get_label_tokenizer()
        self.embedding_dim = embedding_dim

        # self.add_feature_transform_layer will create a transformation layer for each feature
        for feature_key in self.feature_keys:
            input_info = self.dataset.input_info[feature_key]
            self.add_feature_transform_layer(
                feature_key, input_info, special_tokens=["<pad>", "<unk>", "<gap>"]
            )

        # final output layer
        output_size = self.get_output_size(self.label_tokenizer)
        self.bite_net = BiteNet(
            embedding_dim = embedding_dim,
            output_dim = output_size,
            num_heads = num_heads,
            dropout = dropout,
            batch_first = batch_first
        )

        # self.fc = nn.Linear(len(self.feature_keys) * hidden_dim, output_size)

    def forward(self, **kwargs) -> Dict[str, torch.Tensor]:
        patient_emb = []
        patient_mask = []
        for feature_key in self.feature_keys:

            feature_vals = kwargs[feature_key]
            x = self.feat_tokenizers[feature_key].batch_encode_2d(feature_vals, padding=True, truncation=False)
            x = torch.tensor(x, dtype=torch.long, device=self.device)
            pad_idx = self.feat_tokenizers[feature_key].vocabulary("<pad>")

            #create the mask
            mask = (x != pad_idx).bool()

            embeds = self.embeddings[feature_key](x)
            patient_emb.append(embeds)
            patient_mask.append(mask)

        # (patient, features * hidden_dim)
        patient_emb = torch.cat(patient_emb, dim=1)
        patient_mask = ~(torch.cat(patient_mask, dim=1))

        # (patient, label_size)
        logits = self.bite_net(patient_emb, patient_mask)
        print(logits)
        # obtain y_true, loss, y_prob
        y_true = self.prepare_labels(kwargs[self.label_key], self.label_tokenizer)
        loss = self.get_loss_function()(logits, y_true)
        y_prob = self.prepare_y_prob(logits)
        return {"loss": loss, "y_prob": y_prob, "y_true": y_true}

In [56]:
model_dxtx = PyHealthBiteNet(
    dataset = mimic3_dxtx,
    feature_keys = ["conditions", "procedures"],
    label_key = "label",
    mode = "binary",
)

In [57]:
model_dxtx(**next(iter(train_loader_dxtx)))

tensor([[      0.],
        [      0.],
        [      0.],
        [      0.],
        [      0.],
        [      0.],
        [      0.],
        [      0.],
        [      0.],
        [      0.],
        [      0.],
        [      0.],
        [      0.],
        [      0.],
        [      0.],
        [      0.],
        [      0.],
        [      0.],
        [      0.],
        [      0.],
        [      0.],
        [      0.],
        [      0.],
        [      0.],
        [      0.],
        [      0.],
        [      0.],
        [      0.],
        [      0.],
        [      0.],
        [3010752.],
        [      0.]], grad_fn=<ReluBackward0>)


/Users/cyg1122/PycharmProjects/dl4h-final-project/venv/lib/python3.8/site-packages/torch/nn/functional.py:4999: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


{'loss': tensor(94086.6719, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>),
 'y_prob': tensor([[0.5000],
         [0.5000],
         [0.5000],
         [0.5000],
         [0.5000],
         [0.5000],
         [0.5000],
         [0.5000],
         [0.5000],
         [0.5000],
         [0.5000],
         [0.5000],
         [0.5000],
         [0.5000],
         [0.5000],
         [0.5000],
         [0.5000],
         [0.5000],
         [0.5000],
         [0.5000],
         [0.5000],
         [0.5000],
         [0.5000],
         [0.5000],
         [0.5000],
         [0.5000],
         [0.5000],
         [0.5000],
         [0.5000],
         [0.5000],
         [1.0000],
         [0.5000]], grad_fn=<SigmoidBackward0>),
 'y_true': tensor([[1.],
         [0.],
         [1.],
         [0.],
         [0.],
         [0.],
         [1.],
         [0.],
         [0.],
         [1.],
         [0.],
         [0.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         

In [54]:
trainer_dxtx = Trainer(model=model_dxtx)
trainer_dxtx.train(
    train_dataloader=train_loader_dxtx,
    val_dataloader=val_loader_dxtx,
    epochs=5,
    monitor="pr_auc",
)

PyHealthBiteNet(
  (embeddings): ModuleDict(
    (conditions): Embedding(415, 128, padding_idx=0)
    (procedures): Embedding(276, 128, padding_idx=0)
  )
  (linear_layers): ModuleDict()
  (bite_net): BiteNet(
    (code_level_attn): MaskEnc(
      (attention): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
      )
      (fc): Linear(in_features=128, out_features=128, bias=True)
      (relu): ReLU()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (visit_level_attn_forward): MaskEnc(
      (attention): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
      )
      (fc): Linear(in_features=128, out_features=128, bias=True)
      (relu): ReLU()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (visit_level_attn_backward): MaskEnc(
      (attention): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=128,

Epoch 0 / 5:   0%|          | 0/7 [00:00<?, ?it/s]

--- Train epoch-0, step-7 ---
loss: 0.6901
Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]/Users/cyg1122/PycharmProjects/dl4h-final-project/venv/lib/python3.8/site-packages/torch/nn/modules/activation.py:1144: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/transformers/attention.cpp:152.)
  return torch._native_multi_head_attention(
Evaluation: 100%|██████████| 1/1 [00:00<00:00, 112.97it/s]
--- Eval epoch-0, step-7 ---
pr_auc: 0.5417
roc_auc: 0.5000
f1: 0.7027
loss: 0.6931
New best pr_auc score (0.5417) at epoch-0, step-7



Epoch 1 / 5:   0%|          | 0/7 [00:00<?, ?it/s]

/Users/cyg1122/PycharmProjects/dl4h-final-project/venv/lib/python3.8/site-packages/torch/nn/functional.py:4999: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
--- Train epoch-1, step-14 ---
loss: 0.6931
Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]/Users/cyg1122/PycharmProjects/dl4h-final-project/venv/lib/python3.8/site-packages/torch/nn/modules/activation.py:1144: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/transformers/attention.cpp:152.)
  return torch._native_multi_head_attention(
Evaluation: 100%|██████████| 1/1 [00:00<00:00, 98.91it/s]
--- Eval epoch-1, step-14 ---
pr_auc: 0.5417
roc_auc: 0.5000
f1: 0.7027
loss: 0.6931



...

/Users/cyg1122/PycharmProjects/dl4h-final-project/venv/lib/python3.8/site-packages/torch/nn/functional.py:4999: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
--- Train epoch-2, step-21 ---
loss: 0.6931
Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]/Users/cyg1122/PycharmProjects/dl4h-final-project/venv/lib/python3.8/site-packages/torch/nn/modules/activation.py:1144: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/transformers/attention.cpp:152.)
  return torch._native_multi_head_attention(
Evaluation: 100%|██████████| 1/1 [00:00<00:00, 114.17it/s]
--- Eval epoch-2, step-21 ---
pr_auc: 0.5417
roc_auc: 0.5000
f1: 0.7027
loss: 0.6931



Epoch 3 / 5:   0%|          | 0/7 [00:00<?, ?it/s]

/Users/cyg1122/PycharmProjects/dl4h-final-project/venv/lib/python3.8/site-packages/torch/nn/functional.py:4999: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
--- Train epoch-3, step-28 ---
loss: 0.6931
Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]/Users/cyg1122/PycharmProjects/dl4h-final-project/venv/lib/python3.8/site-packages/torch/nn/modules/activation.py:1144: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/transformers/attention.cpp:152.)
  return torch._native_multi_head_attention(
Evaluation: 100%|██████████| 1/1 [00:00<00:00, 98.69it/s]
--- Eval epoch-3, step-28 ---
pr_auc: 0.5417
roc_auc: 0.5000
f1: 0.7027
loss: 0.6931



Epoch 4 / 5:   0%|          | 0/7 [00:00<?, ?it/s]

/Users/cyg1122/PycharmProjects/dl4h-final-project/venv/lib/python3.8/site-packages/torch/nn/functional.py:4999: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
--- Train epoch-4, step-35 ---
loss: 0.6931
Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]/Users/cyg1122/PycharmProjects/dl4h-final-project/venv/lib/python3.8/site-packages/torch/nn/modules/activation.py:1144: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/transformers/attention.cpp:152.)
  return torch._native_multi_head_attention(
Evaluation: 100%|██████████| 1/1 [00:00<00:00, 101.18it/s]
--- Eval epoch-4, step-35 ---
pr_auc: 0.5417
roc_auc: 0.5000
f1: 0.7027
loss: 0.6931
Loaded best model


In [66]:
# option 1: use our built-in evaluation metric
score_dxtx = trainer_dxtx.evaluate(test_loader_dxtx)
print (score_dxtx)

# option 2: use our pyhealth.metrics to evaluate
y_true_dxtx, y_prob_dxtx, loss_dxtx = trainer_dxtx.inference(test_loader_dxtx)
binary_metrics_fn(y_true_dxtx, y_prob_dxtx, metrics=["pr_auc"])

Evaluation: 100%|██████████| 33/33 [00:00<00:00, 367.20it/s]


{'pr_auc': 0.726784508178901, 'roc_auc': 0.685080036129632, 'f1': 0.7090909090909092, 'loss': 0.6331828191424861}


Evaluation: 100%|██████████| 33/33 [00:00<00:00, 363.51it/s]


{'pr_auc': 0.726784508178901}